In [1]:
import psycopg2
from psycopg2.extras import execute_batch
import pandas as pd

In [8]:
# establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    dbname = "dw_cs", 
    user = "postgres", 
    #host= 'localhost',
    host= '172.31.160.1',
    password = "test31415",
    port = 5432
)

# Generic function to create table for partitioning

In [3]:
def create_partition(table, attribute, init_month, end_month, year):
    if end_month==12:
        end_month_part=1
        new_year=year+1
    else:
        end_month_part= str(int(end_month)+1)
        new_year=year
    
    query=f"""CREATE TABLE {table}_m{init_month}_m{end_month}_y{year} PARTITION OF {table}
    FOR VALUES FROM ('{year}-{init_month}-01') TO ('{new_year}-{end_month_part}-01');"""
    return query

# Lineitem on shipdate

In [3]:
with conn.cursor() as cur:
 
#    cur.execute("DELETE FROM lineitem;") 
#    cur.execute("DROP TABLE lineitem;")
    cur.execute("""
CREATE TABLE IF NOT EXISTS public.lineitem (
    l_orderkey INT,
    l_partkey INT,
    l_suppkey INT,
    l_linenumber INT,
    l_quantity DECIMAL,
    l_extendedprice DECIMAL,
    l_discount DECIMAL,
    l_tax DECIMAL,
    l_returnflag CHAR(1),
    l_linestatus CHAR(1),
    o_orderdate DATE,
    l_commitdate DATE,
    l_receiptdate DATE,
    l_shipinstruct CHAR(25),
    l_shipmode CHAR(10),
    l_comment VARCHAR(44)
) PARTITION BY RANGE (l_shipdate);
""")


In [5]:
for year in range(1992,1999):
    for month in range(1,12,3):
        query=create_partition('lineitem', 'l_shipdate', month, month+2, year)
        with conn.cursor() as cursor:
            cursor.execute(query)
            conn.commit()
            print('Partition created')

Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created


In [6]:
insertion_trigger="""
    CREATE OR REPLACE FUNCTION insert_lineitem_partition()
    RETURNS TRIGGER AS $$
    BEGIN
        IF ( NEW.l_shipdate >= '1992-01-01' AND NEW.l_shipdate < '1992-04-01' ) THEN
            INSERT INTO lineitem_m1_m3_1992 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1992-04-01' AND NEW.l_shipdate < '1992-07-01' ) THEN
            INSERT INTO lineitem_m4_m6_1992 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1992-07-01' AND NEW.l_shipdate < '1992-10-01' ) THEN
            INSERT INTO lineitem_m7_m9_1992 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1992-10-01' AND NEW.l_shipdate < '1993-01-01' ) THEN
            INSERT INTO lineitem_m10_m12_1992 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1993-01-01' AND NEW.l_shipdate < '1993-04-01' ) THEN
            INSERT INTO lineitem_m1_m3_1993 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1993-04-01' AND NEW.l_shipdate < '1993-07-01' ) THEN
            INSERT INTO lineitem_m4_m6_1993 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1993-07-01' AND NEW.l_shipdate < '1993-10-01' ) THEN
            INSERT INTO lineitem_m7_m9_1993 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1993-10-01' AND NEW.l_shipdate < '1994-01-01' ) THEN
            INSERT INTO lineitem_m10_m12_1993 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1994-01-01' AND NEW.l_shipdate < '1994-04-01' ) THEN
            INSERT INTO lineitem_m1_m3_1994 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1994-04-01' AND NEW.l_shipdate < '1994-07-01' ) THEN
            INSERT INTO lineitem_m4_m6_1994 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1994-07-01' AND NEW.l_shipdate < '1994-10-01' ) THEN
            INSERT INTO lineitem_m7_m9_1994  VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1994-10-01' AND NEW.l_shipdate < '1995-01-01' ) THEN
            INSERT INTO lineitem_m10_m12_1994  VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1995-01-01' AND NEW.l_shipdate < '1995-04-01' ) THEN
            INSERT INTO lineitem_m1_m3_1995 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1995-04-01' AND NEW.l_shipdate < '1995-07-01' ) THEN
            INSERT INTO lineitem_m4_m6_1995 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1995-07-01' AND NEW.l_shipdate < '1995-10-01' ) THEN
            INSERT INTO lineitem_m7_m9_1995 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1995-10-01' AND NEW.l_shipdate < '1996-01-01' ) THEN
            INSERT INTO lineitem_m10_m12_1995 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1996-01-01' AND NEW.l_shipdate < '1996-04-01' ) THEN
            INSERT INTO lineitem_m1_m3_1996 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1996-04-01' AND NEW.l_shipdate < '1996-07-01' ) THEN
            INSERT INTO lineitem_m4_m6_1996 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1996-07-01' AND NEW.l_shipdate < '1996-10-01' ) THEN
            INSERT INTO lineitem_m7_m9_1996 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1996-10-01' AND NEW.l_shipdate < '1997-01-01' ) THEN
            INSERT INTO lineitem_m10_m12_1996 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1997-01-01' AND NEW.l_shipdate < '1997-04-01' ) THEN
            INSERT INTO lineitem_m1_m3_1997 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1997-04-01' AND NEW.l_shipdate < '1997-07-01' ) THEN
            INSERT INTO lineitem_m4_m6_1997 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1997-07-01' AND NEW.l_shipdate < '1997-10-01' ) THEN
            INSERT INTO lineitem_m7_m9_1997 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1997-10-01' AND NEW.l_shipdate < '1998-01-01' ) THEN
            INSERT INTO lineitem_m10_m12_1997 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1998-01-01' AND NEW.l_shipdate < '1998-04-01' ) THEN
            INSERT INTO lineitem_m1_m3_1998 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1998-04-01' AND NEW.l_shipdate < '1998-07-01' ) THEN
            INSERT INTO lineitem_m4_m6_1998 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1998-07-01' AND NEW.l_shipdate < '1998-10-01' ) THEN
            INSERT INTO lineitem_m7_m9_1998 VALUES (NEW.*);
        ELSIF ( NEW.l_shipdate >= '1998-10-01' AND NEW.l_shipdate < '1998-12-02' ) THEN
            INSERT INTO lineitem_m10_m12_1998 VALUES (NEW.*);
        ELSE
            RAISE EXCEPTION 'Date out of range.  Fix the insert_lineitem_partition() function!';
        END IF;
        RETURN NULL;
    END;
    $$ 
    LANGUAGE plpgsql;


    CREATE TRIGGER insert_lineitem_trigger
    BEFORE INSERT ON lineitem
    FOR EACH ROW EXECUTE FUNCTION insert_lineitem_partition();"""


In [7]:
with conn.cursor() as cur:
    cur.execute(insertion_trigger)

In [10]:
# Define the partition indexes
partition_indexes = [idx for idx in range(0, 7_000_000, 1_000_000)]    # 6e6 is the number of rows in the orders table

# Loop through each file
for i in range(1, 11):
    # Loop through each partition
    for j in range(len(partition_indexes) - 1):
        # Read the CSV file in chunks
        df = pd.read_csv(f"/home/saradsai/data_management/TPC-H/lineitem_8.csv", delimiter="|", header=None, skiprows=partition_indexes[j], nrows=partition_indexes[j + 1] - partition_indexes[j])
        
        # Convert the DataFrame to a list of tuples
        data = [tuple(row) for row in df.to_numpy()]
        print(f"Read {len(data)} rows from partition {j} of file {i}")

        # Insert the data into the database
        conn.rollback()
        with conn.cursor() as cur:
            execute_batch(
                cur,
                "INSERT INTO lineitem (l_orderkey, l_partkey, l_suppkey, l_linenumber, l_quantity, l_extendedprice, l_discount, l_tax, l_returnflag, l_linestatus, l_shipdate, l_commitdate, l_receiptdate, l_shipinstruct, l_shipmode, l_comment) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                data
            )
            conn.commit()

    print(f"Inserted {len(data)} rows in the lineitem table from file {i}")

Read 1000000 rows from partition 0 of file 9
Read 1000000 rows from partition 1 of file 9
Read 1000000 rows from partition 2 of file 9
Read 1000000 rows from partition 3 of file 9
Read 1000000 rows from partition 4 of file 9
Read 999992 rows from partition 5 of file 9
Inserted 999992 rows in the lineitem table from file 9
Read 1000000 rows from partition 0 of file 10
Read 1000000 rows from partition 1 of file 10
Read 1000000 rows from partition 2 of file 10
Read 1000000 rows from partition 3 of file 10
Read 1000000 rows from partition 4 of file 10
Read 999992 rows from partition 5 of file 10
Inserted 999992 rows in the lineitem table from file 10


execution time for the population of the partition:
58 m 24 s+ 1m 56 s + 14 m 49 s

# Orders on orderdate

In [7]:
conn.close()

In [4]:
with conn.cursor() as cur:
    
    cur.execute("DELETE FROM orders;")  
    cur.execute("DROP TABLE orders;")
    cur.execute("""CREATE TABLE IF NOT EXISTS public.orders (
    o_orderkey INT,
    o_custkey INT,
    o_orederstatus CHAR(1),
    o_totalprice DECIMAL,
    o_orderdate DATE,
    o_orederpriority CHAR(15),
    o_clerk CHAR(15),
    o_shippriority INT,
    o_comment VARCHAR(79),
    --CONSTRAINT fk_customer FOREIGN KEY (o_custkey) REFERENCES customer(c_custkey)
)
PARTITION BY RANGE (o_orderdate);
                
""")
    conn.commit()


In [9]:
for year in range(1992,1999):
    for month in range(1,12,3):
        query=create_partition('orders', 'o_orderdate', month, month+2, year)
        with conn.cursor() as cursor:
            cursor.execute(query)
            conn.commit()
            print('Partition created')

Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created
Partition created


In [ ]:
insertion_trigger="""
    CREATE OR REPLACE FUNCTION insert_orders_partition()
    RETURNS TRIGGER AS $$
    BEGIN
        IF ( NEW.o_orderdate >= '1992-01-01' AND NEW.o_orderdate < '1992-04-01' ) THEN
            INSERT INTO orders_m1_m3_1992 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1992-04-01' AND NEW.o_orderdate < '1992-07-01' ) THEN
            INSERT INTO orders_m4_m6_1992 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1992-07-01' AND NEW.o_orderdate < '1992-10-01' ) THEN
            INSERT INTO orders_m7_m9_1992 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1992-10-01' AND NEW.o_orderdate < '1993-01-01' ) THEN
            INSERT INTO orders_m10_m12_1992 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1993-01-01' AND NEW.o_orderdate < '1993-04-01' ) THEN
            INSERT INTO orders_m1_m3_1993 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1993-04-01' AND NEW.o_orderdate < '1993-07-01' ) THEN
            INSERT INTO orders_m4_m6_1993 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1993-07-01' AND NEW.o_orderdate < '1993-10-01' ) THEN
            INSERT INTO orders_m7_m9_1993 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1993-10-01' AND NEW.o_orderdate < '1994-01-01' ) THEN
            INSERT INTO orders_m10_m12_1993 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1994-01-01' AND NEW.o_orderdate < '1994-04-01' ) THEN
            INSERT INTO orders_m1_m3_1994 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1994-04-01' AND NEW.o_orderdate < '1994-07-01' ) THEN
            INSERT INTO orders_m4_m6_1994 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1994-07-01' AND NEW.o_orderdate < '1994-10-01' ) THEN
            INSERT INTO orders_m7_m9_1994  VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1994-10-01' AND NEW.o_orderdate < '1995-01-01' ) THEN
            INSERT INTO orders_m10_m12_1994  VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1995-01-01' AND NEW.o_orderdate < '1995-04-01' ) THEN
            INSERT INTO orders_m1_m3_1995 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1995-04-01' AND NEW.o_orderdate < '1995-07-01' ) THEN
            INSERT INTO orders_m4_m6_1995 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1995-07-01' AND NEW.o_orderdate < '1995-10-01' ) THEN
            INSERT INTO orders_m7_m9_1995 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1995-10-01' AND NEW.o_orderdate < '1996-01-01' ) THEN
            INSERT INTO orders_m10_m12_1995 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1996-01-01' AND NEW.o_orderdate < '1996-04-01' ) THEN
            INSERT INTO orders_m1_m3_1996 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1996-04-01' AND NEW.o_orderdate < '1996-07-01' ) THEN
            INSERT INTO orders_m4_m6_1996 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1996-07-01' AND NEW.o_orderdate < '1996-10-01' ) THEN
            INSERT INTO orders_m7_m9_1996 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1996-10-01' AND NEW.o_orderdate < '1997-01-01' ) THEN
            INSERT INTO orders_m10_m12_1996 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1997-01-01' AND NEW.o_orderdate < '1997-04-01' ) THEN
            INSERT INTO orders_m1_m3_1997 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1997-04-01' AND NEW.o_orderdate < '1997-07-01' ) THEN
            INSERT INTO orders_m4_m6_1997 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1997-07-01' AND NEW.o_orderdate < '1997-10-01' ) THEN
            INSERT INTO orders_m7_m9_1997 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1997-10-01' AND NEW.o_orderdate < '1998-01-01' ) THEN
            INSERT INTO orders_m10_m12_1997 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1998-01-01' AND NEW.o_orderdate < '1998-04-01' ) THEN
            INSERT INTO orders_m1_m3_1998 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1998-04-01' AND NEW.o_orderdate < '1998-07-01' ) THEN
            INSERT INTO orders_m4_m6_1998 VALUES (NEW.*);
        ELSIF ( NEW.o_orderdate >= '1998-07-01' AND NEW.o_orderdate < '1998-10-01' ) THEN
            INSERT INTO orders_m7_m9_1998 VALUES (NEW.*);
        ELSE
            RAISE EXCEPTION 'Date out of range.  Fix the insert_orders_partition() function!';
        END IF;
        RETURN NULL;
    END;
    $$ 
    LANGUAGE plpgsql;


    CREATE TRIGGER insert_orders_trigger
    BEFORE INSERT ON orders
    FOR EACH ROW EXECUTE FUNCTION insert_orders_partition();"""


note: "ELSIF ( NEW.o_orderdate >= '1998-10-01' AND NEW.o_orderdate < '1998-12-02' ) THEN
            INSERT INTO orders_m10_m12_1998 VALUES (NEW.*);"
      has been deleted since o.o_orderdate<1998-08-02

In [10]:
partition_indexes = [i for i in range(0, 16_000_000, 1_000_000)]    # 15e6 is the number of rows in the orders table

for i in range(len(partition_indexes) - 1):
    df = pd.read_csv("/home/saradsai/data_management/TPC-H/orders.csv", delimiter="|", header=None, skiprows=partition_indexes[i], nrows=partition_indexes[i + 1] - partition_indexes[i])

    data = df.to_numpy() 
    print(len(data))

    conn.rollback()
    with conn.cursor() as cur:

        for row in data:
            cur.execute(
                "INSERT INTO orders (o_orderkey, o_custkey, o_orederstatus, o_totalprice, o_orderdate, o_orederpriority, o_clerk, o_shippriority, o_comment) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
                (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8])
                )
        conn.commit()

1000000


KeyboardInterrupt: 

In [ ]:
conn.close()

In [ ]:
def insert_primary(table, attribute, init_month, end_month, year):
    
    query=f"""ALTER TABLE {table}_m{init_month}_m{end_month}_y{year} ADD PRIMARY KEY({attribute});"""
    return query

In [ ]:
for year in range(1992,1999):
    for month in range(1,12,3):
        query=insert_primary('orders', 'o_orderkey', month, month+2, year)
        with conn.cursor() as cursor:
            cursor.execute(query)
            conn.commit()
            print('Primary key created')